In [1]:
#question 1 part a
import numpy as np

#load data
D = np.genfromtxt('lines.csv', delimiter=',', skip_header=1)

x1 = D[:, 0]
y1 = D[:, 3]

#stack points
points = np.column_stack((x1, y1))

#compute centriod
centroid = np.mean(points, axis=0)

#subtract centroid from points
A = points - centroid

#compute SVD
U, S, Vt = np.linalg.svd(A)

#smallest singular vector
a,b = Vt[-1]

#compute c
c = -a*centroid[0] - b*centroid[1]

print(f"Line parameters: a={a}, b={b}, c={c}")

Line parameters: a=0.7735616496467872, b=-0.6337210539312553, c=-3.794192210845812


In [2]:
#part b
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression

D = np.genfromtxt('lines.csv', delimiter=',', skip_header=1)

X_cols = D[:, :3]  # x1, x2, x3
y_cols = D[:, 3:] # y1, y2, y3

X_all = X_cols.flatten().reshape(-1, 1)
y_all = y_cols.flatten()

remaining_X = X_all
remaining_y = y_all

lines = []

for i in range(3):
    ransac = RANSACRegressor(LinearRegression(), min_samples=50, residual_threshold=0.5, random_state=0)

    ransac.fit(remaining_X, remaining_y)
    inlier_mask = ransac.inlier_mask_

    coef = ransac.estimator_.coef_[0]
    intercept = ransac.estimator_.intercept_

    lines.append((coef, intercept))
    
    #reemove inliers for next iteration
    remaining_X = remaining_X[~inlier_mask]
    remaining_y = remaining_y[~inlier_mask]

for i, (m,c) in enumerate(lines):
    print(f"Line {i+1}: y = {m:.4f}x + {c:.4f}")

Line 1: y = 0.6784x + -0.2748
Line 2: y = 0.2356x + -0.1044
Line 3: y = 0.3898x + -1.9842
